# Decision Tree

In [132]:
from math import log
import operator

In [133]:
# Calculate Shannon Entropy
'''def shannonEntro(dataset):
    numEntries = len(dataset)
    labelcounts = {}
    for data in dataset:
        label = data[-1] # label is the last column of the dataset
        if label not in labelcounts.keys():
            labelcounts[label] = 0
        labelcounts[label] += 1
    
    shannonEntro = 0.0
    
    for key in labelcounts:
        prob = float(labelcounts[key])/numEntries
        shannonEntro -= prob*log(prob,2)
    return shannonEntro'''
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: #the the number of unique elements and their occurance
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) #log base 2
    return shannonEnt

In [134]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

In [135]:
myData,labels = createDataSet()
print(myData)

print(shannonEntro(myData)) # higher the Entropy, the more mixed up the data is, then the more possible to improve the model

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
0.9709505944546686


In [136]:
'''def splitDataset(dataset,axis,value):
    sperateDataset = []
    for item in dataset:
        if item[axis] == value:
            reducedVec = item[:axis] # extract the elements before axis
            reducedVec.extend(item[axis+1:]) # add the elements after axis
            sperateDataset.append(reducedVec)
    return sperateDataset'''

def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]     #chop out axis used for splitting
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [137]:
splitDataset(myData,0,0)

[[1, 'no'], [1, 'no']]

In [138]:
'''def chooseBestFeatureToSplit(dataset):
    numFeature = len(dataset[0])-1
    baseEntropy = shannonEntro(dataset)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeature):
        featureList = [feat[i] for feat in dataset] # extract ith feature from each data
        uniqueVals = set(featureList) # eliminate repeated values
        newEntropy = 0.0
        for val in uniqueVals:
            subDataset = splitDataset(dataset, i, val) 
            prob = len(subDataset) / float(len(dataset))
            newEntropy = prob * shannonEntro(subDataset) #
        infoGain = baseEntropy - newEntropy
        
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature'''
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1      #the last column is used for the labels
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):        #iterate over all the features
        featList = [example[i] for example in dataSet]#create a list of all the examples of this feature
        uniqueVals = set(featList)       #get a set of unique values
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)     
        infoGain = baseEntropy - newEntropy     #calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain):       #compare this to the best gain so far
            bestInfoGain = infoGain         #if better than current best, set to best
            bestFeature = i
    return bestFeature                      #returns an integer

In [139]:
chooseBestFeatureToSplit(myData)

0

In [140]:
'''def majorityCount(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(),
     key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]'''

def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [141]:
'''def createTree(dataset, labels):
    classList = [example[-1] for example in dataset] # get all classes
    if classList.count(classList[0]) == len(classList): # if all classes in classList are same, return
        return classList[0]
    if len(dataset[0])==1: 
        return majorityCount(classList)     # if no features to split, do the major count to decide the final class 
    bestFeature = chooseBestFeatureToSplit(dataset)
    print(bestFeature)
    bestFeatureLabel = labels[bestFeature]  
    myTree = {bestFeatureLabel:{}}
    del(sublabels[bestFeature])        # delete the node feature from feature labels
    featureVals = [example[bestFeature] for example in dataset] 
    uniqueVals = set(featureVals)
    
    for value in uniqueVals:
        subLabels = labels[:]  #keep the original labels same every time
        myTree[bestFeatureLabel][value] = createTree(splitDataset(dataset,bestFeature,value), subLabels) 
    return myTree'''

'''def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): 
        return classList[0]#stop splitting when all of the classes are equal
    if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree '''
def createTree(dataSet,labels):
    #获取数据集中的最后一列的类标签，存入classList列表
    classList=[example[-1] for example in dataSet]
    #通过count()函数获取类标签列表中第一个类标签的数目
    #判断数目是否等于列表长度，相同表面所有类标签相同，属于同一类
    if classList.count(classList[0])==len(classList):
        return classList[0]
    #遍历完所有的特征属性，此时数据集的列为1，即只有类标签列
    if len(dataSet[0])==1:
        #多数表决原则，确定类标签
        return majorityCnt(classList)
    #确定出当前最优的分类特征
    bestFeat=chooseBestFeatureToSplit(dataSet)
    #在特征标签列表中获取该特征对应的值
    bestFeatLabel=labels[bestFeat]
    #采用字典嵌套字典的方式，存储分类树信息
    myTree={bestFeatLabel:{}}
    
    ##此位置书上写的有误，书上为del(labels[bestFeat])
    ##相当于操作原始列表内容，导致原始列表内容发生改变
    ##按此运行程序，报错'no surfacing'is not in list
    ##以下代码已改正
    
    #复制当前特征标签列表，防止改变原始列表的内容
    subLabels=labels[:]
    #删除属性列表中当前分类数据集特征
    del(subLabels[bestFeat])
    #获取数据集中最优特征所在列
    featValues=[example[bestFeat] for example in dataSet]
    #采用set集合性质，获取特征的所有的唯一取值
    uniqueVals=set(featValues)
    #遍历每一个特征取值
    for value in uniqueVals:
        #采用递归的方法利用该特征对数据集进行分类
        #@bestFeatLabel 分类特征的特征标签值
        #@dataSet 要分类的数据集
        #@bestFeat 分类特征的标称值
        #@value 标称型特征的取值
        #@subLabels 去除分类特征后的子特征标签列表
        myTree[bestFeatLabel][value]=createTree(splitDataSet\
            (dataSet,bestFeat,value),subLabels)
    return myTree

In [142]:
myTree = createTree(myData, labels)
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

# Classification function for an existing decision tree

In [147]:
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree.keys())[0] # first feature name
    secondDict = inputTree[firstStr] 
    featIndex = featLabels.index(firstStr) #index of first feature
    
    for key in secondDict.keys(): 
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__=='dict':
                classLabel = classify(secondDict[key],featLabels,testVec)
            else:
                classLabel = secondDict[key]
    return classLabel
        

In [149]:
classify(myTree,labels,[1,1])

'yes'

# Persisting Decision Tree with "pickle"

In [154]:
def storeTree(inputTree,filename):
    import pickle
    fw = open(filename,'wb') # write as byte
    pickle.dump(inputTree,fw)
    fw.close()
    
def grabTree(filename):
    import pickle 
    fr = open(filename,'rb') # read as byte
    return pickle.load(fr)

In [155]:
storeTree(myTree, 'classifierStorage.txt')
grabTree('classifierStorage.txt')

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

# Using DecisionTree Predicting contact lens type

In [156]:
fr = open('lenses.txt')
lenses = [inst.strip().split('\t') for inst in fr.readlines()]
lensesLabels = ['age','prescript','astigmatic','tearRate']
lensesTree = createTree(lenses,lensesLabels)
lensesTree

{'tearRate': {'normal': {'astigmatic': {'no': {'age': {'pre': 'soft',
      'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}},
      'young': 'soft'}},
    'yes': {'prescript': {'hyper': {'age': {'pre': 'no lenses',
        'presbyopic': 'no lenses',
        'young': 'hard'}},
      'myope': 'hard'}}}},
  'reduced': 'no lenses'}}